In [ ]:
# simsiam
!experiment_name="debug";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=0,1,2 torchrun --nproc_per_node=3 --master_port=1234 train_multi_GPU.py \
    --wandb False --wandb_model dryrun --sync_bn False --amp True --aux False \
    --model_name aspp_contrast_resnet101 --pre_trained deeplabv3_resnet101_coco.pth \
    --weight_only_backbone False \
    --data_path pascal-voc-2012 --num_classes 21 \
    --epochs 40 --batch_size 8 --batch_size_val 8 --memory_size 0 \
    --contrast 0 --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --loss_name aspp_loss --sample self_pace_epochs \
    --name_date $name_date \
    2>&1 | tee $dir_log